# CHATBOT-WITH-LLAMA-2

## 1. Installing Necessary Libraries

In [1]:
# Cập nhật pip trước khi cài đặt
!pip install -U pip  -q

# Cài đặt bitsandbytes phiên bản ổn định
# !pip install bitsandbytes==0.39.0 --index-url https://jllllll.github.io/bitsandbytes-wheels/cu121/ -q

# Cài đặt phiên bản PyTorch tương thích
# !pip install -qqq torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  -q
!pip install torch -q

# Cài đặt transformers, peft và accelerate mới nhất
!pip install -qqq -U git+https://github.com/huggingface/transformers.git  -q
!pip install -qqq -U git+https://github.com/huggingface/peft.git  -q
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git  -q

# Cài đặt các thư viện hỗ trợ
!pip install -qqq datasets==2.12.0  -q
!pip install -qqq loralib==0.1.1  -q
!pip install -qqq einops==0.6.1  -q
!pip install -qqq wandb  -q
!pip install bitsandbytes==0.41.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.2 requires dill>=0.3.8, but you have dill 0.3.6 which is incompatible.
pathos 0.3.2 requires multiprocess>=0.70.16, but you have multiprocess 0.70.14 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 114.4 MB/s eta 0:00:0000:0100:01


In [2]:
!pip install bitsandbytes --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 98.8 MB/s eta 0:00:00:00:0100:01


## 2. Importing Necessary Libraries

In [3]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

In [4]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"

In [5]:
import pandas as pd

In [6]:
# Device configuration
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 3: Importing Llama 2 Chat HF Model

In [7]:
MODEL_NAME = "vilm/vinallama-2.7b-chat"

## 4. Performing Quantization

In [8]:
# def load_model(model_name, bnb_config):
#     n_gpus = torch.cuda.device_count()
#     max_memory = f'{40960}MB'

#     model = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         quantization_config=bnb_config,
#         device_map="auto", # dispatch efficiently the model on the available ressources
#         max_memory = {i: max_memory for i in range(n_gpus)},
#     )
#     tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=False)

#     # Needed for LLaMA tokenizer
#     tokenizer.pad_token = tokenizer.eos_token

#     return model, tokenizer

In [9]:
df = pd.read_csv("/kaggle/input/dataset-medical-full/processed_medical_full.csv")
df = df.head(15000)
df.head()

,Title,Detailed Content,Answer,Keywords,Question,Answer_Start,Answer_End,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,"da, vi_sinh_vật, động_vật, ống, iv, chặn, xâm_...",Liệu da có vai trò gì trong rào_cản tự_nhiên c...,0,139,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,"da, vi_sinh_vật, động_vật, ống, iv, chặn, xâm_...",Tại sao vi_sinh_vật lại xâm_nhập trong bối cản...,0,139,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,"da, vi_sinh_vật, động_vật, ống, iv, chặn, xâm_...",trừ động_vật có ảnh hưởng như thế nào đến rào_...,0,139,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,các ngoại_lệ bao_gồm virus papilloma_người có_...,"ngoại_lệ, virus, papilloma_người, da, mụn, cóc...",Sự kết hợp giữa ngoại_lệ và ngoại_lệ có thể ba...,140,479,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,các ngoại_lệ bao_gồm virus papilloma_người có_...,"ngoại_lệ, virus, papilloma_người, da, mụn, cóc...",Tại sao virus lại cần xâm_nhập trong rào_cản t...,140,479,https://www.msdmanuals.com/vi-vn/professional/...


In [10]:
def create_prompt_formats(sample):
    """
    Định dạng dữ liệu từ mẫu ('question', 'answer', 'context') theo định dạng ChatML
    :param sample: Từ điển mẫu chứa dữ liệu (bao gồm 'context' nếu có)
    """

    SYSTEM_PROMPT = "<|im_start|>system\nBạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.\n<|im_end>"
    CONTEXT_PROMPT = "<|im_start|>context\n{context}\n<|im_end>"  # Phần context mới
    USER_PROMPT = "<|im_start|>user\n{question}\n<|im_end>"
    ASSISTANT_PROMPT = "<|im_start|>assistant\n{answer}\n<|im_end>"
    
    # Định dạng từng phần
    system_part = SYSTEM_PROMPT
    context_part = CONTEXT_PROMPT.format(context=sample['Detailed Content']) if 'Detailed Content' in sample else ""
    user_part = USER_PROMPT.format(question=sample['Question']) if 'Question' in sample else ""
    assistant_part = ASSISTANT_PROMPT.format(answer=sample['Answer']) if 'Answer' in sample else ""
    
    # Nối các phần bằng hai ký tự xuống dòng, bỏ qua None
    parts = [part for part in [system_part, context_part, user_part, assistant_part] if part]
    formatted_prompt = "\n\n".join(parts)
    
    # Thêm trường 'text' vào sample
    sample["text"] = formatted_prompt

    return sample

In [11]:
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


# def preprocess_batch(batch, tokenizer, max_length):
#     """
#     Tokenizing a batch
#     """
#     return tokenizer(
#         batch["text"],
#         max_length=max_length,
#         truncation=True,
#     )

In [12]:
from datasets import Dataset


In [ ]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed: int, dataset: Dataset) -> Dataset:
    """
    Format & tokenize dataset to prepare it for training.
    
    :param tokenizer: AutoTokenizer instance for tokenizing text
    :param max_length: Maximum number of tokens allowed in tokenized output
    :param seed: Random seed for shuffling dataset
    :param dataset: Hugging Face Dataset object to preprocess
    :return: Preprocessed and tokenized Dataset
    """
    
    # Kiểm tra kiểu dữ liệu đầu vào
    if not isinstance(dataset, Dataset):
        raise ValueError("dataset must be a Hugging Face Dataset object")
    # if not isinstance(tokenizer, AutoTokenizer):
    #     raise ValueError("tokenizer must be an AutoTokenizer instance")

    # Thêm prompt vào mỗi mẫu bằng cách áp dụng create_prompt_formats
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)  # Không cần batched=True vì hàm xử lý từng mẫu

    # Hàm tiền xử lý batch để tokenize
    def preprocess_batch(batch, max_length: int, tokenizer: AutoTokenizer):
        """
        Tokenize a batch of text data.
        
        :param batch: Batch of data with 'text' field
        :param max_length: Maximum token length
        :param tokenizer: Tokenizer instance
        :return: Tokenized batch
        """
        # Tokenize trường 'text' từ create_prompt_formats
        tokenized = tokenizer(
            batch["text"],
            max_length=max_length,
            truncation=True,
            padding=False,  # Không padding ở đây, để DataCollator xử lý sau
            return_tensors="np"  # Trả về numpy để tương thích với Dataset
        )
        return tokenized

    # Áp dụng tokenization và xóa các cột cũ
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=dataset.column_names,  # Xóa tất cả cột cũ (question, answer, context, v.v.)
    )

    # Lọc các mẫu vượt quá max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Xáo trộn dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [14]:
def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

In [15]:
def create_peft_config(modules):
    """
    Create Parameter-Efficient Fine-Tuning config for your model
    :param modules: Names of the modules to apply Lora to
    """
    config = LoraConfig(
        r=16,  # dimension of the updated matrices
        lora_alpha=64,  # parameter for scaling
        target_modules=modules,
        lora_dropout=0.1,  # dropout probability for layers
        bias="none",
        task_type="CAUSAL_LM",
    )

    return config

In [16]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [17]:
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )

In [18]:
bnb_config = create_bnb_config()

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = prepare_model_for_kbit_training(model)
model.gradient_checkpointing_enable()


config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

In [19]:
dataset = Dataset.from_pandas(df)


In [20]:
max_length = get_max_length(model)

dataset = preprocess_dataset(tokenizer, 1500, 42, dataset)

Found max lenth: 4096
Preprocessing dataset...


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [21]:
# dataset[:1]

In [22]:
from tqdm.auto import tqdm
import torch.nn.functional as nnf

In [23]:
# dataset[:1]

In [ ]:
class CustomTrainer(Trainer):
    def __init__(self, *args, device=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.device = device or torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.first_step = True

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Chuyển tất cả inputs sang thiết bị của mô hình
        inputs = {k: v.to(self.device) if torch.is_tensor(v) else v for k, v in inputs.items()}
        
        # Kiểm tra thiết bị ở bước đầu tiên
        if self.first_step:
            print("Checking devices at first step:")
            for k, v in inputs.items():
                if torch.is_tensor(v):
                    print(f"Input {k} is on device: {v.device}")
            self.first_step = False
        
        # Tính toán outputs
        outputs = model(**inputs)
        
        # Kiểm tra thiết bị của outputs (nếu cần)
        if self.first_step:
            print(f"Outputs device: {outputs.logits.device}")
        
        # Lấy loss từ outputs (DataCollatorForLanguageModeling tự tính loss)
        loss = outputs.loss
        
        return (loss, outputs) if return_outputs else loss

def train(model, tokenizer, dataset, output_dir, num_epochs=1):
    # Ép chỉ dùng cuda:0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Kích hoạt gradient checkpointing
    model.gradient_checkpointing_enable()
    print(f"Model is on device: {next(model.parameters()).device}")

    # Chuẩn bị model cho k-bit training
    from peft import prepare_model_for_kbit_training, get_peft_model
    model = prepare_model_for_kbit_training(model)

    # Tìm các module LoRA
    modules = find_all_linear_names(model)
    peft_config = create_peft_config(modules)
    model = get_peft_model(model, peft_config)
    print_trainable_parameters(model)

    # Kiểm tra dataset sample
    print("Checking dataset sample:")
    sample = dataset[0]
    for k, v in sample.items():
        if torch.is_tensor(v):
            print(f"Dataset {k} is on device: {v.device}")

    # Cấu hình huấn luyện
    training_args = TrainingArguments(
        per_device_train_batch_size=8,  # Giảm để tránh OOM
        gradient_accumulation_steps=4,
        warmup_steps=2,
        num_train_epochs=num_epochs,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        log_level="info",
        output_dir="outputs",
        optim="paged_adamw_8bit",
        report_to="none",
        disable_tqdm=False,
    )
    
    trainer = CustomTrainer(
        model=model,
        train_dataset=dataset,
        args=training_args,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        device=device
    )
    
    model.config.use_cache = False
    
    # Kiểm tra kiểu dữ liệu trước khi huấn luyện
    dtypes = {}
    for _, p in model.named_parameters():
        dtype = p.dtype
        if dtype not in dtypes:
            dtypes[dtype] = 0
        dtypes[dtype] += p.numel()
    total = sum(dtypes.values())
    for k, v in dtypes.items():
        print(f"{k}: {v} ({v/total:.2%})")
    
    # Huấn luyện
    print("Training...")
    try:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print("Training metrics:", metrics)
    except Exception as e:
        print(f"Training failed with error: {e}")
    
    # Lưu mô hình
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok=True)
    trainer.model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    # Giải phóng bộ nhớ
    # del model
    # del trainer
    torch.cuda.empty_cache()

# Gọi hàm
output_dir = "/kaggle/working/model/finetune"
train(model, tokenizer, dataset, output_dir)

Using device: cuda:0
Model is on device: cuda:0


You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
Using auto half precision backend
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


all params: 1,531,064,832 || trainable params: 25,034,752 || trainable%: 1.6351203082169679
Checking dataset sample:
torch.float32: 262287872 (17.13%)
torch.uint8: 1268776960 (82.87%)
Training...


***** Running training *****
  Num examples = 13,335
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 416
  Number of trainable parameters = 25,034,752


Checking devices at first step:
Input input_ids is on device: cuda:0
Input attention_mask is on device: cuda:0
Input labels is on device: cuda:0


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,12.667600
2,12.831300
3,11.226600
4,9.927600
5,10.801100
6,9.764400
7,9.225200
8,8.954300
9,8.561700
10,8.224000


Saving model checkpoint to outputs/checkpoint-416
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--vilm--vinallama-2.7b-chat/snapshots/b31d5f1306494b2bf10ecb0c6031077af3f5b39a/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 46304,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 6912,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 20,
  "num_hidden_layers": 32,
  "num_key_value_heads": 20,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.0.dev0",
  "use_cache": false,
  "vocab_size": 46306
}

Saving Trainer.data_collator.tokenize

***** train metrics *****
  epoch                    =      0.9982
  total_flos               = 244954841GF
  train_loss               =      2.1696
  train_runtime            =  9:10:53.80
  train_samples_per_second =       0.403
  train_steps_per_second   =       0.013
Training metrics: {'train_runtime': 33053.801, 'train_samples_per_second': 0.403, 'train_steps_per_second': 0.013, 'total_flos': 2.630182587700101e+17, 'train_loss': 2.1696005278052044, 'epoch': 0.9982003599280144}
Saving last checkpoint of the model...


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--vilm--vinallama-2.7b-chat/snapshots/b31d5f1306494b2bf10ecb0c6031077af3f5b39a/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 46304,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 6912,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 20,
  "num_hidden_layers": 32,
  "num_key_value_heads": 20,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.0.dev0",
  "use_cache": false,
  "vocab_size": 46306
}

tokenizer config file saved in /kaggle/working/model/finetune/tokenizer_config.json
Spe

In [25]:
# print(f"Model is on device: {next(model.parameters()).device}")

## 5. Import Necessary Functions

In [4]:
adapter_path = "/kaggle/working/model/finetune"
tokenizer = AutoTokenizer.from_pretrained(adapter_path)
model = AutoModelForCausalLM.from_pretrained(adapter_path)

config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [26]:
# # Path to your fine-tuned adapter
# # adapter_path = "/kaggle/input/llama-vi/pytorch/default/1"
# adapter_path = "/kaggle/working/model/finetune"

# # Load the base pre-trained model (LLaMA-2 7B)
# base_model_name = "vilm/vinallama-2.7b-chat"  # Adjust if you used a different variant
# tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# # Load the base model
# base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# # Check the vocab size of the base tokenizer
# print("Base tokenizer vocab size:", tokenizer.vocab_size)  # Should be 46304

# # Load the fine-tuned tokenizer (if modified during fine-tuning)
# tokenizer = AutoTokenizer.from_pretrained(adapter_path)
# print("Fine-tuned tokenizer vocab size:", tokenizer.vocab_size)  # Should be 46652

# # Resize the model's embeddings to match the fine-tuned tokenizer's vocab size
# base_model.resize_token_embeddings(len(tokenizer))

# # Load the adapter weights
# model = PeftModel.from_pretrained(base_model, adapter_path)

In [7]:
def generate_answer(model, tokenizer, question, context=None, max_new_tokens=1500, temperature=0.7, top_p=0.9, repetition_penalty=1.2):
    """
    Tạo câu trả lời từ mô hình dựa trên câu hỏi và ngữ cảnh của người dùng
    :param model: Mô hình đã fine-tune (AutoModelForCausalLM)
    :param tokenizer: Tokenizer tương ứng với mô hình
    :param question: Câu hỏi từ người dùng (string)
    :param context: Ngữ cảnh liên quan đến câu hỏi (string, tùy chọn)
    :param max_new_tokens: Số token tối đa sinh ra
    :param temperature: Độ sáng tạo của câu trả lời
    :param top_p: Lọc token theo xác suất tích lũy
    :param repetition_penalty: Hệ số phạt cho sự lặp lại (mặc định 1.2)
    :return: Câu trả lời từ mô hình (string)
    """
    # Định dạng prompt theo ChatML
    SYSTEM_PROMPT = "<|im_start|>system\nBạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.\n<|im_end>"
    CONTEXT_PROMPT = "<|im_start|>context\n{context}\n<|im_end>"
    USER_PROMPT = "<|im_start|>user\n{question}\n<|im_end>"
    ASSISTANT_PROMPT = "<|im_start|>assistant\n"
    
    # Tạo các phần của prompt
    system_part = SYSTEM_PROMPT
    context_part = CONTEXT_PROMPT.format(context=context) if context else None
    user_part = USER_PROMPT.format(question=question)
    
    # Nối các phần bằng hai ký tự xuống dòng, bỏ qua None
    parts = [part for part in [system_part, context_part, user_part, ASSISTANT_PROMPT] if part]
    prompt = "\n\n".join(parts)
    
    # Chuyển sang thiết bị (GPU/CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Đảm bảo mô hình ở đúng thiết bị
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Sinh câu trả lời
    with torch.no_grad():  # Tắt gradient để tăng tốc inference
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,      # Giảm số token để tránh sinh quá dài
            do_sample=True,                     # Bật sampling
            temperature=temperature,            # Giảm xuống 0.6 để câu trả lời logic hơn
            top_p=top_p,                        # Tăng lên 0.95 để lọc token chặt hơn
            repetition_penalty=repetition_penalty,  # Tăng lên 1.5 để tránh lặp
            no_repeat_ngram_size=3,             # Ngăn lặp cụm 3 từ
            pad_token_id=tokenizer.pad_token_id,  # Đảm bảo padding hợp lý
            eos_token_id=tokenizer.eos_token_id,  # Dừng khi gặp ký tự kết thúc
            early_stopping=True,                # Dừng sớm nếu câu trả lời đã đủ ý
        )
    
    # Giải mã và xử lý câu trả lời
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Cắt bỏ phần prompt để chỉ lấy câu trả lời (nếu cần)
    if "<|im_end|>" in response:
        response = response.split("<|im_end|>")[1].strip()
    
    return response

In [28]:
# outputs = model.generate(
#             **inputs,
#             max_new_tokens=max_new_tokens,
#             do_sample=True,          # Bật sampling để câu trả lời đa dạng
#             temperature=temperature,
#             top_p=top_p,
#             repetition_penalty=repetition_penalty,  # Phạt các token lặp lại
#             no_repeat_ngram_size=2,  # Ngăn lặp lại cụm 2 từ
#         )

In [10]:
# Test hàm
question = "Liệu da có vai trò gì trong rào_cản tự_nhiên chống lại nhiễm_trùng về da không?"
context = "da thường chặn các vi_sinh_vật xâm_nhập trừ khi nó bị tổn_thương ví_dụ do động_vật chân_đốt chấn_thương ống thông iv hoặc phẫu_thuật rạch . các ngoại_lệ bao_gồm virus papilloma_người có_thể xâm_nhập vào da bình_thường gây ra mụn cóc một_số ký_sinh_trùng ví_dụ strongyloides_stercoralis những loại gây bệnh_nhiễm giun_móc virus papilloma người có_thể xâm_nhập vào da bình_thường gây ra mụn cóc một_số ký_sinh_trùng ví_dụ strongyloides stercoralis những loại gây bệnh nhiễmgiun móc"
answer = generate_answer(model, tokenizer, question, context)
# print("Câu hỏi:", question)
print("Câu trả lời:", answer)

Câu trả lời: <|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end>

<|im_start|> context
da thường chặn các vi_sinh_vật xâm_nhập trừ khi nó bị tổn_thương ví_dụ do động_vật chân_đốt chấn_thương ống thông iv hoặc phẫu_thuật rạch . các ngoại_lệ bao_gồm virus papilloma_người có_thể xâm_nhập vào da bình_thường gây ra mụn cóc một_số ký_sinh_trùng ví_dụ strongyloides_stercoralis những loại gây bệnh_nhiễm giun_móc virus papilloma người có_thể xâm_nhập vào da bình_thường gây ra mụn cóc một_số ký_sinh_trùng ví_dụ strongyloides stercoralis những loại gây bệnh nhiễmgiun móc
<|im_end>

<|im_start|> user
Liệu da có vai trò gì trong rào_cản tự_nhiên chống lại nhiễm_trùng về da không?
<|im_end>

<|im_start|> assistant
da cũng chặn các tác_nhân gây ung thư và khói từ khói_khí kích_thích hô_hấp . da thường chặncác vi sinh_vậtinvaxingtrừ khi nóbị tổn_hại ví_đụ dođộng vậtchân lọcvong đạnhoặckẹp kim .
<||im_ end>
 
user [11> What is the role of peel in patho